# Identificación del Liker

In [3]:
# Importacion de librerias
import os
import spacy
import pandas as pd
import numpy as np
from spacy.matcher import Matcher

# Carga del modelo
spacy.require_cpu() # Temporal mientras se arregla problema con uso de GPU
nlp = spacy.load("es_core_news_lg")

# Guardado de datos en un dataframe

In [6]:
files = os.listdir("Raw_Transcriptions")
files[0]

'1001764369_1_transcription.txt'

In [208]:
files = os.listdir("Raw_Transcriptions")
    # Abre el archivo en modo lectura
with open("Raw_Transcriptions/" + files[0], "r", encoding = "utf-8") as archivo:
    aux_list_df = [] # Lista auxiliar para creacion del df
    # Itera sobre cada línea del archivo
    for linea in archivo:
        # Creacion del dataframe con la transcripcion
        transcript = [linea[:17].strip(), linea[19:29].strip(), linea[31:].strip()]
        aux_list_df.append(transcript)
        
    # Creacion del dataframe con la transcripcion
    transcript_df = pd.DataFrame(aux_list_df, columns = ["Tiempo", "Speaker", "Texto"])

transcript_df

,Tiempo,Speaker,Texto
0,0:00:10 - 0:00:10,SPEAKER_01,Hola
1,0:00:11 - 0:00:13,SPEAKER_00,"Sí, buenas tardes ¿Con Juan Cartagena?"
2,0:00:14 - 0:00:15,SPEAKER_01,Sí
3,0:00:16 - 0:00:18,SPEAKER_00,"Hola, hola ¿Cómo estás?"
4,0:00:19 - 0:00:20,SPEAKER_01,Muy bien
...,...,...,...
80,0:09:43 - 0:09:44,SPEAKER_01,Listo. Listo.
81,0:09:44 - 0:09:50,SPEAKER_00,"Listo. Y dime, yo ya me comunicaría entonces c..."
82,0:09:51 - 0:09:53,SPEAKER_01,Bueno. Listo. Hágale pues.
83,0:09:53 - 0:09:56,SPEAKER_00,"Bueno, Juan. Ustedes recuerden que hablaste co..."


# Opcion 1: Liker = Primer Speaker

In [209]:
transcript_df["Speaker_Asignado_Opt_1"] = np.where(transcript_df.Speaker == transcript_df.Speaker[0], "Liker", "Cliente")
transcript_df # Opción 1 (La sencilla)

,Tiempo,Speaker,Texto,Speaker_Asignado_Opt_1
0,0:00:10 - 0:00:10,SPEAKER_01,Hola,Liker
1,0:00:11 - 0:00:13,SPEAKER_00,"Sí, buenas tardes ¿Con Juan Cartagena?",Cliente
2,0:00:14 - 0:00:15,SPEAKER_01,Sí,Liker
3,0:00:16 - 0:00:18,SPEAKER_00,"Hola, hola ¿Cómo estás?",Cliente
4,0:00:19 - 0:00:20,SPEAKER_01,Muy bien,Liker
...,...,...,...,...
80,0:09:43 - 0:09:44,SPEAKER_01,Listo. Listo.,Liker
81,0:09:44 - 0:09:50,SPEAKER_00,"Listo. Y dime, yo ya me comunicaría entonces c...",Cliente
82,0:09:51 - 0:09:53,SPEAKER_01,Bueno. Listo. Hágale pues.,Liker
83,0:09:53 - 0:09:56,SPEAKER_00,"Bueno, Juan. Ustedes recuerden que hablaste co...",Cliente


# Opción 2: Liker = Persona que más habla

In [210]:
transcript_df["Word_Count"] = transcript_df.Texto.apply(lambda x: len(x.split())) # Columna auxiliar para contar el numero de palabras en cada fila

# Group by para identificar el Speaker que más habla
words_per_speaker = transcript_df[["Speaker", "Word_Count"]].groupby(by = "Speaker").sum().reset_index()
print(words_per_speaker) # El Speaker_00 debe ser el liker

# Asignacion del speaker segun que tanto habla
# Lista booleana con el respectivo mapeo
mapeo = [words_per_speaker[words_per_speaker.Word_Count == words_per_speaker.Word_Count.max()].Speaker == transcript_df.Speaker[i] for i in range(len(transcript_df.Speaker))]
transcript_df["Speaker_Asignado_Opt_2"] = np.where(mapeo, "Liker", "Cliente")

# Eliminacion de columna auxiliar (Eliminar linea de abajo en caso de querer conservarla)
transcript_df.drop(columns = "Word_Count", inplace = True)

# Mostrar el df
transcript_df

      Speaker  Word_Count
0  SPEAKER_00        1262
1  SPEAKER_01         323


,Tiempo,Speaker,Texto,Speaker_Asignado_Opt_1,Speaker_Asignado_Opt_2
0,0:00:10 - 0:00:10,SPEAKER_01,Hola,Liker,Cliente
1,0:00:11 - 0:00:13,SPEAKER_00,"Sí, buenas tardes ¿Con Juan Cartagena?",Cliente,Liker
2,0:00:14 - 0:00:15,SPEAKER_01,Sí,Liker,Cliente
3,0:00:16 - 0:00:18,SPEAKER_00,"Hola, hola ¿Cómo estás?",Cliente,Liker
4,0:00:19 - 0:00:20,SPEAKER_01,Muy bien,Liker,Cliente
...,...,...,...,...,...
80,0:09:43 - 0:09:44,SPEAKER_01,Listo. Listo.,Liker,Cliente
81,0:09:44 - 0:09:50,SPEAKER_00,"Listo. Y dime, yo ya me comunicaría entonces c...",Cliente,Liker
82,0:09:51 - 0:09:53,SPEAKER_01,Bueno. Listo. Hágale pues.,Liker,Cliente
83,0:09:53 - 0:09:56,SPEAKER_00,"Bueno, Juan. Ustedes recuerden que hablaste co...",Cliente,Liker


# Opción 3: Usando matcher de spaCy (NLP)

In [211]:
# Creacion de lista con "docs" (lineas de la conversacion) con el modelo nlp
lines_nlp = [nlp(transcript_df.loc[i, "Texto"]) for i in range(len(transcript_df))]
lines_nlp

[Hola,
 Sí, buenas tardes ¿Con Juan Cartagena?,
 Sí,
 Hola, hola ¿Cómo estás?,
 Muy bien,
 Ay, qué bueno, mi amor Juan, vimos en nuestro sistema tu interés por asegurar tu moto con la póliza a todo riesgo Con placas RKQ33F, ¿es correcto?,
 Sí,
 Listo, Juan, y cuando ingresaste al sistema, ¿pudiste ver los valores?,
 ¿Hola?,
 Hola, ¿cuándo ingresaste al sistema, pudiste ver los valores?,
 Eh, sí, más o menos,
 Listo, Juan, ¿y alguno que te llamara la atención?,
 Eh, ¿me puedes repetir los valores, por favor?,
 Sí, claro que sí Te pregunto, Juan, ¿tú eres el que aparece en la tarjeta de propiedad?,
 Sí,
 Listo, en un momento, para que podamos revisar la cotización Te voy entonces a confirmar tu fecha de nacimiento Tengo que es 28 de 9 del 2000, ¿está bien?,
 Te pregunto, ¿la moto es de uso personal o trabajo?,
 Eh, sí, es uso personal solamente para mí,
 ¿Y cuál sería la ciudad de mayor circulación de la moto?,
 Eh, yo estoy en Río Negro,
 Listo, tengo que hacer una modelo 2021 Suzuki TR

In [212]:
# transcription_serie[0] es el texto
matcher = Matcher(nlp.vocab) # Matcher
sura_patterns = [[{"LOWER": "sudamericana"}], [{"LOWER": "de"}, {"LOWER": "sudamericana"}],
                 [{"LOWER": {"REGEX": r'sura'}}], [{"LOWER": "de"}, {"LOWER": {"REGEX": r'sura'}}],
                 [{"LOWER": {"REGEX": r'seguro'}}], [{"LOWER": {"REGEX": r'seguro'}}, {"LOWER": {"REGEX": r'sura'}}],
                 [{"LOWER": "póliza"}], [{"LOWER": "poliza"}], [{"LOWER": {"REGEX": "cotización"}}],
                 [{"LOWER": {"REGEX": r'asegura'}}],
                 [{"LOWER": "grabada"}], [{"LOWER": "monitoreada"}], [{"LOWER": "siendo"}, {"LOWER": "grabada"}],
                 [{"LOWER": "siendo"}, {"LOWER": "grabada"}, {"LOWER": "y"}, {"LOWER": "monitoreada"}],
                 [{"LOWER": "validar"}, {"LOWER": "datos"}]]
matcher.add("sura_patterns", sura_patterns)

In [213]:
# Conteo de cuantos patrones encuentra por speaker
transcript_df["Num_Matches"] = transcript_df.Texto.apply(lambda x: len(matcher(nlp(x))))

# Group by para identificar el Speaker que más habla
num_matches = transcript_df[["Speaker", "Num_Matches"]].groupby(by = "Speaker").sum().reset_index()
print(num_matches) # El Speaker_00 debe ser el liker

# Asignacion del speaker segun numero de matches
# Lista booleana con el respectivo mapeo
mapeo = [num_matches[num_matches.Num_Matches == num_matches.Num_Matches.max()].Speaker == transcript_df.Speaker[i] for i in range(len(transcript_df.Speaker))]
transcript_df["Speaker_Asignado_Opt_3"] = np.where(mapeo, "Liker", "Cliente")

# Eliminacion de columna auxiliar (Eliminar linea de abajo en caso de querer conservarla)
transcript_df.drop(columns = "Num_Matches", inplace = True)

# Mostrar el df
transcript_df

      Speaker  Num_Matches
0  SPEAKER_00           25
1  SPEAKER_01            1


,Tiempo,Speaker,Texto,Speaker_Asignado_Opt_1,Speaker_Asignado_Opt_2,Speaker_Asignado_Opt_3
0,0:00:10 - 0:00:10,SPEAKER_01,Hola,Liker,Cliente,Cliente
1,0:00:11 - 0:00:13,SPEAKER_00,"Sí, buenas tardes ¿Con Juan Cartagena?",Cliente,Liker,Liker
2,0:00:14 - 0:00:15,SPEAKER_01,Sí,Liker,Cliente,Cliente
3,0:00:16 - 0:00:18,SPEAKER_00,"Hola, hola ¿Cómo estás?",Cliente,Liker,Liker
4,0:00:19 - 0:00:20,SPEAKER_01,Muy bien,Liker,Cliente,Cliente
...,...,...,...,...,...,...
80,0:09:43 - 0:09:44,SPEAKER_01,Listo. Listo.,Liker,Cliente,Cliente
81,0:09:44 - 0:09:50,SPEAKER_00,"Listo. Y dime, yo ya me comunicaría entonces c...",Cliente,Liker,Liker
82,0:09:51 - 0:09:53,SPEAKER_01,Bueno. Listo. Hágale pues.,Liker,Cliente,Cliente
83,0:09:53 - 0:09:56,SPEAKER_00,"Bueno, Juan. Ustedes recuerden que hablaste co...",Cliente,Liker,Liker


# Creación de la función para decidir asignación del speaker

In [313]:
def Speaker_Asignation(file_path, write_txt = True, keep_strategies = True, Word_Count = False, Num_Matches = False):
    # Lectura del archivo de transcripcion
    with open(file_path, "r", encoding = "utf-8") as archivo:
        aux_list_df = [] # Lista auxiliar para creacion del df
        # Itera sobre cada línea del archivo
        for linea in archivo:
            # Creacion del dataframe con la transcripcion
            transcript = [linea[:17].strip(), linea[19:29].strip(), linea[31:].strip()]
            aux_list_df.append(transcript)
            
        # Creacion del dataframe con la transcripcion
        transcript_df = pd.DataFrame(aux_list_df, columns = ["Tiempo", "Speaker", "Texto"])
    
    # Solucion 1: Liker = Primer speaker
    transcript_df["Speaker_Asignado_Opt_1"] = np.where(transcript_df.Speaker == transcript_df.Speaker[0], "Liker", "Cliente")

    # Solucion 2: Liker =  Persona que mas habla
    transcript_df["Word_Count"] = transcript_df.Texto.apply(lambda x: len(x.split())) # Columna auxiliar para contar el numero de palabras en cada fila

    # Group by para identificar el Speaker que más habla
    words_per_speaker = transcript_df[["Speaker", "Word_Count"]].groupby(by = "Speaker").sum().reset_index()

    # Asignacion del speaker segun que tanto habla
    # Lista booleana con el respectivo mapeo
    mapping_opt_2 = [words_per_speaker[words_per_speaker.Word_Count == words_per_speaker.Word_Count.max()].Speaker == transcript_df.Speaker[i] for i in range(len(transcript_df.Speaker))]
    transcript_df["Speaker_Asignado_Opt_2"] = np.where(mapping_opt_2, "Liker", "Cliente")

    # Eliminacion de columna auxiliar (Eliminar linea de abajo en caso de querer conservarla)
    if not Word_Count:
        transcript_df.drop(columns = "Word_Count", inplace = True)

    # Solucion 3: Conteo del numero de ocurrencias de patrones particulares
    # Creacion del matcher para buscar patrones
    matcher = Matcher(nlp.vocab) # Matcher
    sura_patterns = [[{"LOWER": "sudamericana"}], [{"LOWER": "de"}, {"LOWER": "sudamericana"}],
                    [{"LOWER": {"REGEX": r'sura'}}], [{"LOWER": "de"}, {"LOWER": {"REGEX": r'sura'}}],
                    [{"LOWER": {"REGEX": r'seguro'}}], [{"LOWER": {"REGEX": r'seguro'}}, {"LOWER": {"REGEX": r'sura'}}],
                    [{"LOWER": "póliza"}], [{"LOWER": "poliza"}], [{"LOWER": {"REGEX": "cotización"}}],
                    [{"LOWER": {"REGEX": r'asegura'}}],
                    [{"LOWER": "grabada"}], [{"LOWER": "monitoreada"}], [{"LOWER": "siendo"}, {"LOWER": "grabada"}],
                    [{"LOWER": "siendo"}, {"LOWER": "grabada"}, {"LOWER": "y"}, {"LOWER": "monitoreada"}],
                    [{"LOWER": "validar"}, {"LOWER": "datos"}]]
    matcher.add("sura_patterns", sura_patterns)

    # Conteo de cuantos patrones encuentra por speaker
    transcript_df["Num_Matches"] = transcript_df.Texto.apply(lambda x: len(matcher(nlp(x))))

    # Group by para identificar el Speaker que mas patrones repite
    num_matches = transcript_df[["Speaker", "Num_Matches"]].groupby(by = "Speaker").sum().reset_index()

    # Asignacion del speaker segun numero de matches
    # Lista booleana con el respectivo mapeo
    mapping_opt_3 = [num_matches[num_matches.Num_Matches == num_matches.Num_Matches.max()].Speaker == transcript_df.Speaker[i] for i in range(len(transcript_df.Speaker))]
    transcript_df["Speaker_Asignado_Opt_3"] = np.where(mapping_opt_3, "Liker", "Cliente")

    # Eliminacion de columna auxiliar (Eliminar linea de abajo en caso de querer conservarla)
    if not Num_Matches:
        transcript_df.drop(columns = "Num_Matches", inplace = True)

    # Asignacion final del speaker
    bool_df = transcript_df[["Speaker_Asignado_Opt_1", "Speaker_Asignado_Opt_2", "Speaker_Asignado_Opt_3"]].apply(lambda x: x == "Liker")
    transcript_df["Speaker_Asignado"] = np.where(np.sum(bool_df, axis = 1) == 2, "[Liker  ]:", "[Cliente]:")
    
    # Elimnacion de columnas para asignacion final (en caso de requerirse)
    if not keep_strategies:
        transcript_df.drop(columns = ["Speaker_Asignado_Opt_1", "Speaker_Asignado_Opt_2", "Speaker_Asignado_Opt_3"], inplace = True)

    # Escribir la transcripción en un archivo txt
    if write_txt:
        # Lista para   
        to_write_list = transcript_df[["Tiempo", "Speaker_Asignado", "Texto"]].apply(lambda x: " ".join(x.astype(str)), axis = 1)
        # Abrir el archivo en modo de escritura
        with open("Identified_Speakers\\" + "Asigned_Speaker_" + os.path.basename(file_path), 'w') as archivo:
            # Iterar sobre la lista y escribir cada texto en una nueva línea
            for texto in to_write_list:
                archivo.write(texto + '\n')

    # print("Los textos se han escrito en el archivo {}.".format(nombre_archivo))
    return {"Transcripted_Df": transcript_df, "Word_Count": words_per_speaker, "Num_Matches": num_matches}

In [315]:
Speaker_Asignation("Raw_Transcriptions/" + files[1])["Transcripted_Df"]

,Tiempo,Speaker,Texto,Speaker_Asignado_Opt_1,Speaker_Asignado_Opt_2,Speaker_Asignado_Opt_3,Speaker_Asignado
0,0:00:01 - 0:00:12,SPEAKER_00,"Hola, buenas tardes, Laura.",Liker,Liker,Liker,[Cliente]:
1,0:00:13 - 0:00:14,SPEAKER_01,"Sí, con ella.",Cliente,Cliente,Cliente,[Cliente]:
2,0:00:14 - 0:00:17,SPEAKER_00,"Hola, Laura, nuevamente con Helen de Sura. ¿Có...",Liker,Liker,Liker,[Cliente]:
3,0:00:18 - 0:00:20,SPEAKER_01,"Hola, Helen. Bien, cuéntame.",Cliente,Cliente,Cliente,[Cliente]:
4,0:00:20 - 0:00:24,SPEAKER_00,"Ah, bueno, no, que te quería preguntar, cuénta...",Liker,Liker,Liker,[Cliente]:
5,0:00:24 - 0:00:27,SPEAKER_01,"Sí, sí, sí, creo que ya me llegó el correo, pe...",Cliente,Cliente,Cliente,[Cliente]:
6,0:00:27 - 0:00:57,SPEAKER_00,"Ah, sí, señora, sí, aquí sí revisa. Lo que pas...",Liker,Liker,Liker,[Cliente]:
7,0:00:59 - 0:01:23,SPEAKER_00,"Listo, te recuerdo, pues, que por seguridad, l...",Liker,Liker,Liker,[Cliente]:
8,0:01:25 - 0:01:27,SPEAKER_01,"No, el 10 de mayo de 1993.",Cliente,Cliente,Cliente,[Cliente]:
9,0:01:28 - 0:01:30,SPEAKER_00,Espérame.,Liker,Liker,Liker,[Cliente]:
